# Preliminaries for Using Marlowe without Demeter.Run

This notebook provides instructions on setting up one's environment for running the Marlowe examples in this starter kit. It covers the following information:

- Marlowe tools
- Environment variables
- Deploying Marlowe Runtime
- Creating addresses and signing keys
    - The faucet
    - The lender
    - The borrower
- Obtaining test ada
- Fund the addresses of the parties
    - Using Daedalus or a web-browser wallet
    - Using a local faucet at the command line

## Marlowe Tools

Three alternative workflows are available for running Marlowe contracts:
1. Marlowe CLI (`marlowe-cli`) for lightweight experiments with Marlowe transactions.
2. Marlowe Runtime CLI (`marlowe`) for non-web applications that use the Marlowe Runtime backend services.
3. Marlowe Runtime Web (`marlowe-web-server`) for web applications that use Marlowe Runtime backend services.

Marlowe Runtime provides a variety of transaction-building, UTxO management, querying, and submission services for using Marlowe contracts: this makes it easy to run Marlowe contracts without attending to the details of the Cardano ledger and Plutus smart contracts. On the contrary, Marlowe CLI does not support querying and UTxO management, so it is best suited for experienced Cardano developers.

![Tools for Running and Querying Marlowe Contracts](images/marlowe-tools.png)

## Environment Variables

The various Marlowe tools use [environment variables](https://en.wikipedia.org/wiki/Environment_variable) to specify the values of parameters and the network locations of services. Only the enviornment variables for the particular tool(s) being used need be set; the others may be left unset or ignored. The following table summarizes the tools' use of these settings.

| Workflow            | Tool          | Environment Variable        | Typical Value | Description                                                                                     |
|---------------------|---------------|-----------------------------|---------------|-------------------------------------------------------------------------------------------------|
| Marlowe CLI         | `marlowe-cli` | `CARDANO_NODE_SOCKET_PATH`  | `node.socket` | Location of the socket for the `cardano-node` service.                                          |
|                     |               | `CARDANO_TESTNET_MAGIC`     | 2             | The "magic number" for the Cardano testnet being used, or not set if `mainnet` is being used.   |
| Marlowe Runtime CLI | `marlowe`     | `MARLOWE_RT_HOST`           | `127.0.0.1`   | The host machine's IP address for Marlowe Runtime.                                              |
|                     |               | `MARLOWE_RT_PORT`           | `3700`        | The port number for the `marlowe-proxy` service on the Marlowe Runtime host machine.            |
| Marlowe Runtime Web | `curl` etc.   | `MARLOWE_RT_WEBSERVER_HOST` | `127.0.0.1`   | The host machine's IP address for Marlowe Runtime's web server.                                 |
|                     |               | `MARLOWE_RT_WEBSERVER_PORT` | `8080`        | The port number for the `marlowe-web-server` service on the Marlowe Runtime web server machine. |

## Deploying Marlowe Runtime

Marlowe Runtime consists of several backend services and work together with a web server.

![The architecture of Marlowe Runtime](images/runtime-architecture.png)

First, make sure that either [docker-compose](https://docs.docker.com/compose/install/) or [podman-compose](https://github.com/containers/podman-compose#readme) is installed on your system.

This repository contains a [git submodule](https://git-scm.com/book/en/v2/Git-Tools-Submodules) that contains the docker-compose configuration for running a Cardano Node and Marlowe Runtime. Initialize this submodule.

In [1]:
git submodule update --init --recursive

Submodule 'marlowe-compose' (git@github.com:bwbush/marlowe-compose.git) registered for path 'marlowe-compose'
Cloning into '/extra/iohk/marlowe-starter-kit/marlowe-compose'...
Submodule path 'marlowe-compose': checked out '19fe4263350212537f93e53b2297ca81772d8edc'


Select either the public Cardano testnet pre-production (`preprod`) or preview (`preview`). The `preprod` network is easiest to start with because it contains fewer transactions than `preview`, which makes synchronize the Cardano Node and the Marlowe Runtime indexers much faster.

In [2]:
export NETWORK=preprod

Now start the docker containers. Here we use `podman-compose`, but the `docker-compose` works identically.

In [3]:
podman-compose -f marlowe-compose/compose.yaml up -d

['podman', '--version', '']
using podman version: 4.3.1
** excluding:  set()
podman volume inspect marlowe-compose_shared || podman volume create marlowe-compose_shared
['podman', 'volume', 'inspect', 'marlowe-compose_shared']
['podman', 'volume', 'create', '--label', 'io.podman.compose.project=marlowe-compose', '--label', 'com.docker.compose.project=marlowe-compose', 'marlowe-compose_shared']
['podman', 'volume', 'inspect', 'marlowe-compose_shared']
podman volume inspect marlowe-compose_node-db || podman volume create marlowe-compose_node-db
['podman', 'volume', 'inspect', 'marlowe-compose_node-db']
['podman', 'volume', 'create', '--label', 'io.podman.compose.project=marlowe-compose', '--label', 'com.docker.compose.project=marlowe-compose', 'marlowe-compose_node-db']
['podman', 'volume', 'inspect', 'marlowe-compose_node-db']
['podman', 'network', 'exists', 'marlowe-compose_default']
podman run --name=marlowe-compose_node_1 -d --label io.podman.compose.config-hash=123 --label io.podman

Check to make sure that all of the services were created and have the status "Up".

In [4]:
podman-compose -f marlowe-compose/compose.yaml ps

['podman', '--version', '']
using podman version: 4.4.2
podman ps -a --filter label=io.podman.compose.project=marlowe-compose
CONTAINER ID  IMAGE                                               COMMAND     CREATED             STATUS                       PORTS                   NAMES
c9ea7fe655a3  docker.io/inputoutput/cardano-node:1.35.4                       About a minute ago  Up About a minute (healthy)                          marlowe-compose_node_1
0bf114667144  docker.io/library/postgres:11.5-alpine              postgres    About a minute ago  Up About a minute (healthy)                          marlowe-compose_postgres_1
41749da8a001  docker.io/iohkbuild/marlowe:chain-indexer-20230302              About a minute ago  Up About a minute                                    marlowe-compose_chain-indexer_1
ee9fd794fb8d  docker.io/iohkbuild/marlowe:chain-sync-20230302                 About a minute ago  Up About a minute                                    marlowe-compose_chain-sync_1
2f

Use the command `podman-compose -f marlowe-compose/compose.yaml stop` to stop the services and the command `podman-compose -f marlowe-compose/compose.yaml start` to restart them. The command `podman-compose -f marlowe-compose/compose.yaml down` will stop the services and destroy the data folders the resources used by the services: unless external volumes were used when starting the services, the Cardano blockchain and Marlowe database will be removed.

Now set the environment variables for the Cardano and Marlowe Runtime services.

In [5]:
if command -v podman > /dev/null
then
  DOCKER_CLI=podman
else
  DOCKER_CLI=docker
fi

# Only required for `marlowe-cli` and `cardano-cli`.
export CARDANO_NODE_SOCKET_PATH="$($DOCKER_CLI volume inspect marlowe-compose_shared | jq -r '.[0].Mountpoint')/node.socket"
case "$NETWORK" in
  preprod)
    export CARDANO_TESTNET_MAGIC=1
    ;;
  preview)
    export CARDANO_TESTNET_MAGIC=2
    ;;
  *)
    # Use `mainnet` as the default.
    unset CARDANO_TESTNET_MAGIC
    ;;
esac

# Only required for `marlowe`.
export MARLOWE_RT_HOST="127.0.0.1"
export MARLOWE_RT_PORT=3700

# Only required for REST API.
export MARLOWE_RT_WEBSERVER_HOST="127.0.0.1"
export MARLOWE_RT_WEBSERVER_PORT=3780
export MARLOWE_RT_WEBSERVER_URL="http://$MARLOWE_RT_WEBSERVER_HOST:$MARLOWE_RT_WEBSERVER_PORT"

When `cardano-node` is started, it may take minutes or hours to synchronize with the tip of the blockchain, depending upon the network. Wait until querying the node shows that its `syncProgress` is 100%.

In [6]:
cardano-cli query tip --testnet-magic "$CARDANO_TESTNET_MAGIC" | json2yaml

block: 731649
epoch: 57
era: Babbage
hash: ad9ed62386341677ada848804618b6198c1d4187a0bfbd008500d2ef9595943b
slot: 23307149
syncProgress: '100.00'


Check that the `marlowe` command can communicate with the Marlowe Runtime backend services by querying the history of one of the Marlowe contracts that has previously been executed on the blockchain.

In [7]:
marlowe log "f06e4b760f2d9578c8088ea5289ba6276e68ae3cbaf5ad27bcfc77dc413890db#1"

transaction f06e4b760f2d9578c8088ea5289ba6276e68ae3cbaf5ad27bcfc77dc413890db (creation)
ContractId:      f06e4b760f2d9578c8088ea5289ba6276e68ae3cbaf5ad27bcfc77dc413890db#1
SlotNo:          11630746
BlockNo:         224384
BlockId:         f729f57b3bd99dd1d55a5a65b8c74f459dadae784dd55536444770dd2c2cdd2e
ScriptAddress:   addr_test1wp4f8ywk4fg672xasahtk4t9k6w3aql943uxz5rt62d4dvqu3c6jv
Marlowe Version: 1



transaction 5a3ed57653b4635c76d2949558f3718e34324a8a1ffc740360ae7d85839de6d9 (close)
ContractId: f06e4b760f2d9578c8088ea5289ba6276e68ae3cbaf5ad27bcfc77dc413890db#1
SlotNo:     16674281
BlockNo:    458964
BlockId:    23adcef9d89afc4354155c62960512c810619d8244239f8c642745b034a58fc2
Inputs:     []




Check that one can communicate with the Marlowe Runtime web server and receive a `200 OK` response.

In [8]:
curl -sSI "$MARLOWE_RT_WEBSERVER_URL/healthcheck"

HTTP/1.1 200 OK
Date: Thu, 16 Mar 2023 18:12:51 GMT
Server: Warp/3.3.24
Content-Type: application/json;charset=utf-8



## Creating Addresses and Signing Keys

The [Cardano Developers Portal](https://developers.cardano.org/docs/stake-pool-course/handbook/keys-addresses/) contains instructions for creating addresses and signing keys.

This starter kit uses the following addresses:
- An ***optional*** local *Faucet* used to fund parties to Marlowe contracts.
- The *Lender* party for the examples in this starter kit.
- The *Borrower* party for the examples in this starter kit.

The instructions below detail how to create signing keys and addresses for these parties. It is assumed that one has the signing key and address for the faucet and that the faucet is already funded with test ada.

### The Faucet

***This step is optional if one is using a wallet already funded with test ada.***

Set the file names for this party's signing key and verification key.

In [9]:
FAUCET_SKEY=keys/faucet.skey
FAUCET_VKEY=keys/faucet.vkey

Generate the keys if they haven't already been generated.

In [10]:
if [[ ! -e "$FAUCET_SKEY" ]]
then
  cardano-cli address key-gen \
    --signing-key-file "$FAUCET_SKEY" \
    --verification-key-file "$FAUCET_VKEY"
fi

Compute the faucet's address on the testnet.

In [11]:
FAUCET_ADDR=$(cardano-cli address build --testnet-magic "$CARDANO_TESTNET_MAGIC" --payment-verification-key-file "$FAUCET_VKEY" )
if [[ ! -e keys/faucet.address ]]
then
  echo "$FAUCET_ADDR" > keys/faucet.address
fi
echo "FAUCET_ADDR = $FAUCET_ADDR"

FAUCET_ADDR = addr_test1vq9prvx8ufwutkwxx9cmmuuajaqmjqwujqlp9d8pvg6gupczgtm9j


### The Lender

Set the file names for this party's signing key and verification key.

In [12]:
LENDER_SKEY=keys/lender.skey
LENDER_VKEY=keys/lender.vkey

Generate the keys if they haven't already been generated.

In [13]:
if [[ ! -e "$LENDER_SKEY" ]]
then
  cardano-cli address key-gen \
    --signing-key-file "$LENDER_SKEY" \
    --verification-key-file "$LENDER_VKEY"
fi

Compute the party's address on the testnet.

In [14]:
LENDER_ADDR=$(cardano-cli address build --testnet-magic "$CARDANO_TESTNET_MAGIC" --payment-verification-key-file "$LENDER_VKEY" )
if [[ ! -e keys/lender.address ]]
then
  echo "$LENDER_ADDR" > keys/lender.address
fi
echo "LENDER_ADDR = $LENDER_ADDR"

LENDER_ADDR = addr_test1vqd3yrtjyx49uld43lvwqaf7z4k03su8gf2x4yr7syzvckgfzm4ck


### The Borrower

Set the file names for this party's signing key and verification key.

In [15]:
BORROWER_SKEY=keys/borrower.skey
BORROWER_VKEY=keys/borrower.vkey

Generate the keys if they haven't already been generated.

In [16]:
if [[ ! -e "$BORROWER_SKEY" ]]
then
  cardano-cli address key-gen \
    --signing-key-file "$BORROWER_SKEY"  \
    --verification-key-file "$BORROWER_VKEY"
fi

Compute the party's address on the testnet.

In [17]:
BORROWER_ADDR=$(cardano-cli address build --testnet-magic "$CARDANO_TESTNET_MAGIC" --payment-verification-key-file "$BORROWER_VKEY" )
if [[ ! -e keys/borrower.address ]]
then
  echo "$BORROWER_ADDR" > keys/borrower.address
fi
echo "BORROWER_ADDR = $BORROWER_ADDR"

BORROWER_ADDR = addr_test1vpy4n4peh4suv0y55yptur0066j5kds8r4ncnuzm0vpzfgg0dhz6d


### The Mediator

Set the file names for this party's signing key and verification key.

In [18]:
MEDIATOR_SKEY=keys/mediator.skey
MEDIATOR_VKEY=keys/mediator.vkey

Generate the keys if they haven't already been generated.

In [19]:
if [[ ! -e "$MEDIATOR_SKEY" ]]
then
  cardano-cli address key-gen \
    --signing-key-file "$MEDIATOR_SKEY"  \
    --verification-key-file "$MEDIATOR_VKEY"
fi

Compute the party's address on the testnet.

In [20]:
MEDIATOR_ADDR=$(cardano-cli address build --testnet-magic "$CARDANO_TESTNET_MAGIC" --payment-verification-key-file "$MEDIATOR_VKEY" )
if [[ ! -e keys/mediator.address ]]
then
  echo "$MEDIATOR_ADDR" > keys/mediator.address
fi
echo "MEDIATOR_ADDR = $MEDIATOR_ADDR"

MEDIATOR_ADDR = addr_test1vr6tytqs3x8qgewhw89m3xrz58t3tqu2hfsecw0u06lf3hg052wsv


## Obtaining Test Ada

In order to execute transactions on a Cardano network, one needs the native currency ada to pay fees and use as funds. There are the faucets for the public testnets at https://docs.cardano.org/cardano-testnet/tools/faucet where one can obtain test ada daily.

Optionally, it can be convenient to centrally manage funds with the [Daedalus wallet](https://docs.cardano.org/cardano-testnet/daedalus-testnet) or one of the [web-browser wallets](https://builtoncardano.com/ecosystem/wallets): be sure to select the correct public testnet if using one of these wallets.

If you will be using a local faucet, then send test ada to the faucet address created in the previous section. Otherwise, send the test ada to the Daedalus or web-browser wallet.

In [21]:
echo "FAUCET_ADDR = $FAUCET_ADDR"

FAUCET_ADDR = addr_test1vq9prvx8ufwutkwxx9cmmuuajaqmjqwujqlp9d8pvg6gupczgtm9j


## Fund the Addresses of the Parties

We'll fund each address with 1000 test ada.

In [22]:
echo "LENDER_ADDR = $LENDER_ADDR"
echo "BORROWER_ADDR = $BORROWER_ADDR"
echo "MEDIATOR_ADDR = $MEDIATOR_ADDR"

LENDER_ADDR = addr_test1vqd3yrtjyx49uld43lvwqaf7z4k03su8gf2x4yr7syzvckgfzm4ck
BORROWER_ADDR = addr_test1vpy4n4peh4suv0y55yptur0066j5kds8r4ncnuzm0vpzfgg0dhz6d
MEDIATOR_ADDR = addr_test1vr6tytqs3x8qgewhw89m3xrz58t3tqu2hfsecw0u06lf3hg052wsv


### Using Daedalus or a Web-Browser Wallet

If you already have a wallet wallet that contains test ada, then you can just send the funds to the addresses of the keys that we created in the previous section. The screenshot below shows using Daedalus to fund the lender address.

![Sending funds with Daedalus](images/daedalus-example.png)

### Using a Local Faucet at the Command Line

One can use `cardano-cli` or `marlowe-cli` send funds to an address. Here we use `marlowe-cli`.

In [23]:
# Note that `FAUCET_ADDR` must have already been funded with test ada.

# 1 ada = 1,000,000 lovelace
ADA=1000000

# Send 1000 ada
AMOUNT=$((1000 * ADA))

# Execute the transaction.
marlowe-cli util fund-address \
 --lovelace "$AMOUNT" \
 --out-file /dev/null \
 --source-wallet-credentials "$FAUCET_ADDR:$FAUCET_SKEY" \
 --submit 600 \
 "$LENDER_ADDR" "$BORROWER_ADDR" "$MEDIATOR_ADDR"

TxId "dd3a06e118f9ccfdc6d8721779228d1730a606cf183489e679f2584b4445d883"


See that the addresses have indeed been funded:

In [24]:
echo
echo "Lender @ $LENDER_ADDR"
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$LENDER_ADDR"
echo

echo
echo "Borrower @ $BORROWER_ADDR"
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$BORROWER_ADDR"
echo
echo
echo "Mediator @ $MEDIATOR_ADDR"
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$MEDIATOR_ADDR"
echo


Lender @ addr_test1vqd3yrtjyx49uld43lvwqaf7z4k03su8gf2x4yr7syzvckgfzm4ck
                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
dd3a06e118f9ccfdc6d8721779228d1730a606cf183489e679f2584b4445d883     1        1000000000 lovelace + TxOutDatumNone


Borrower @ addr_test1vpy4n4peh4suv0y55yptur0066j5kds8r4ncnuzm0vpzfgg0dhz6d
                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
dd3a06e118f9ccfdc6d8721779228d1730a606cf183489e679f2584b4445d883     2        1000000000 lovelace + TxOutDatumNone


Mediator @ addr_test1vr6tytqs3x8qgewhw89m3xrz58t3tqu2hfsecw0u06lf3hg052wsv
                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
dd3a06e118f9ccfdc6d8721779